In [303]:
%pylab inline
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.font_manager import FontProperties
import collections
import seaborn as sns

Populating the interactive namespace from numpy and matplotlib


/home/cmb-panasas2/wenzhenl/miniconda3/envs/raptor/lib/python3.5/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['f']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [286]:
# fins = ['/home/cmb-panasas2/wenzhenl/benchmark/uorfs/human/SRP010679_human/SRX118286_translating_ORFs.tsv',
#            '/home/cmb-panasas2/wenzhenl/benchmark/uorfs/human/SRP063852_human/SRX1254413_translating_ORFs.tsv',
#            '/home/cmb-panasas2/wenzhenl/benchmark/uorfs/human/SRP098789_human/SRX2536421_translating_ORFs.tsv',
#            '/home/cmb-panasas2/wenzhenl/benchmark/uorfs/human/SRP102021_human/SRX2647167_translating_ORFs.tsv']

fins = ['/home/cmb-panasas2/wenzhenl/benchmark/uorfs/mouse/SRP003554_mouse/SRX026871_translating_ORFs.tsv',
           '/home/cmb-panasas2/wenzhenl/benchmark/uorfs/mouse/SRP062407_mouse/SRX1149649_translating_ORFs.tsv',
           '/home/cmb-panasas2/wenzhenl/benchmark/uorfs/mouse/SRP078005_mouse/SRX1900396_translating_ORFs.tsv',
           '/home/cmb-panasas2/wenzhenl/benchmark/uorfs/mouse/SRP091889_mouse/SRX2255510_translating_ORFs.tsv',
           '/home/cmb-panasas2/wenzhenl/benchmark/uorfs/mouse/SRP115915_mouse/SRX3110803_translating_ORFs.tsv']

In [287]:
cds = {}
uorfs = {}
start_codons = {}

In [288]:
for fin in fins:
    df = pd.read_table(fin, header=0, sep='\t', usecols=['ORF_ID', 'ORF_type', 'status', 'length', 'transcript_id', 'gene_id', 'gene_name', 'start_codon'])
    a = df.loc[(df['ORF_type'] == 'annotated') & (df['status'] == 'translating'), 'gene_id'].tolist()
    cds[fin] = set(a)
    b = df.loc[(df['ORF_type'] == 'super_uORF') & (df['status'] == 'translating'), 'gene_id'].tolist()
    uorfs[fin] = set(b)
    s = df.loc[(df['ORF_type'] == 'super_uORF') & (df['status'] == 'translating'), 'start_codon'].tolist()
    start_codons[fin] = s

In [289]:
[len(cds[x]) for x in cds]

[14542, 10564, 11264, 10369, 12739]

In [272]:
joint = (cds['/home/cmb-panasas2/wenzhenl/benchmark/uorfs/human/SRP010679_human/SRX118286_translating_ORFs.tsv'] 
            & cds['/home/cmb-panasas2/wenzhenl/benchmark/uorfs/human/SRP063852_human/SRX1254413_translating_ORFs.tsv']
            & cds['/home/cmb-panasas2/wenzhenl/benchmark/uorfs/human/SRP098789_human/SRX2536421_translating_ORFs.tsv']
            & cds['/home/cmb-panasas2/wenzhenl/benchmark/uorfs/human/SRP102021_human/SRX2647167_translating_ORFs.tsv'])

In [273]:
len(joint)

6979

In [301]:
[(x, len(uorfs[x]) / len(cds[x])) for x in uorfs]

[('/home/cmb-panasas2/wenzhenl/benchmark/uorfs/mouse/SRP062407_mouse/SRX1149649_translating_ORFs.tsv',
  0.25409159675422915),
 ('/home/cmb-panasas2/wenzhenl/benchmark/uorfs/mouse/SRP078005_mouse/SRX1900396_translating_ORFs.tsv',
  0.1155812192351382),
 ('/home/cmb-panasas2/wenzhenl/benchmark/uorfs/mouse/SRP115915_mouse/SRX3110803_translating_ORFs.tsv',
  0.177734375),
 ('/home/cmb-panasas2/wenzhenl/benchmark/uorfs/mouse/SRP003554_mouse/SRX026871_translating_ORFs.tsv',
  0.2724467161732086),
 ('/home/cmb-panasas2/wenzhenl/benchmark/uorfs/mouse/SRP091889_mouse/SRX2255510_translating_ORFs.tsv',
  0.26022450741816466)]

In [304]:
# joint = (uorfs['/home/cmb-panasas2/wenzhenl/benchmark/uorfs/human/SRP010679_human/SRX118286_translating_ORFs.tsv'] 
#             | uorfs['/home/cmb-panasas2/wenzhenl/benchmark/uorfs/human/SRP063852_human/SRX1254413_translating_ORFs.tsv']
#             |  uorfs['/home/cmb-panasas2/wenzhenl/benchmark/uorfs/human/SRP098789_human/SRX2536421_translating_ORFs.tsv']
#             |  uorfs['/home/cmb-panasas2/wenzhenl/benchmark/uorfs/human/SRP102021_human/SRX2647167_translating_ORFs.tsv'])

In [296]:
joint = (uorfs['/home/cmb-panasas2/wenzhenl/benchmark/uorfs/mouse/SRP003554_mouse/SRX026871_translating_ORFs.tsv']
           | uorfs['/home/cmb-panasas2/wenzhenl/benchmark/uorfs/mouse/SRP062407_mouse/SRX1149649_translating_ORFs.tsv']
           | uorfs['/home/cmb-panasas2/wenzhenl/benchmark/uorfs/mouse/SRP078005_mouse/SRX1900396_translating_ORFs.tsv']
           | uorfs['/home/cmb-panasas2/wenzhenl/benchmark/uorfs/mouse/SRP091889_mouse/SRX2255510_translating_ORFs.tsv']
           | uorfs['/home/cmb-panasas2/wenzhenl/benchmark/uorfs/mouse/SRP115915_mouse/SRX3110803_translating_ORFs.tsv'])

In [299]:
valid = {x for x in joint if sum([x in uorfs[y] for y in uorfs]) > 2}

In [300]:
with open('mouse_uorf_three_samples.txt', 'w') as fout:
    for g in valid:
        fout.write('{}\n'.format(g[:g.rfind('.')]))

In [261]:
joint = (uorfs['/home/cmb-panasas2/wenzhenl/benchmark/uorfs/human/SRP063852_human/SRX1254413_translating_ORFs.tsv']
            &  uorfs['/home/cmb-panasas2/wenzhenl/benchmark/uorfs/human/SRP098789_human/SRX2536421_translating_ORFs.tsv']
            &  uorfs['/home/cmb-panasas2/wenzhenl/benchmark/uorfs/human/SRP102021_human/SRX2647167_translating_ORFs.tsv'])

In [262]:
len(joint)

427

In [302]:
for f in start_codons:
    print(collections.Counter(start_codons[f]))

Counter({'CTG': 2968, 'AGG': 2647, 'GTG': 2075, 'AAG': 1736, 'TTG': 1232, 'ATG': 1034, 'ACG': 898, 'ATT': 844, 'ATC': 844, 'ATA': 374})
Counter({'CTG': 704, 'AGG': 698, 'GTG': 570, 'AAG': 444, 'TTG': 316, 'ACG': 236, 'ATG': 208, 'ATC': 204, 'ATT': 186, 'ATA': 106})
Counter({'CTG': 1258, 'AGG': 1164, 'GTG': 968, 'AAG': 709, 'TTG': 542, 'ACG': 413, 'ATG': 377, 'ATC': 333, 'ATT': 321, 'ATA': 151})
Counter({'CTG': 1912, 'AGG': 1788, 'GTG': 1478, 'AAG': 1072, 'TTG': 883, 'ACG': 692, 'ATT': 511, 'ATC': 507, 'ATG': 484, 'ATA': 250})
Counter({'CTG': 2101, 'AGG': 1951, 'GTG': 1636, 'AAG': 1249, 'TTG': 966, 'ACG': 718, 'ATG': 617, 'ATC': 544, 'ATT': 479, 'ATA': 290})


# candidate analysis

In [218]:
a = df.loc[df['ORF_type'] == 'annotated', 'gene_id'].tolist()

In [219]:
len(a)

58776

In [220]:
b = df.loc[df['ORF_type'] == 'novel', 'gene_id'].tolist()

In [221]:
len(b)

770478

In [222]:
len(set(a))

22566

In [223]:
len(set(b))

32488

In [224]:
len(set(b)) / len(set(a))

1.4396880262341576

# translation analysis

In [225]:
a = df.loc[(df['ORF_type'] == 'annotated') & (df['status'] == 'translating'), 'gene_id'].tolist()

In [226]:
len(a)

30891

In [227]:
b = df.loc[(df['ORF_type'] == 'novel') & (df['status'] == 'translating'), 'gene_id'].tolist()

In [228]:
len(b)

71234

In [229]:
len(set(a))

11264

In [230]:
len(set(b))

7186

In [212]:
len(set(b)) / len(set(a))

0.177734375

In [213]:
len(set(a) & set(b))

1854

In [214]:
len(set(a) - set(b))

9410

In [215]:
len(set(b) - set(a))

148

In [238]:
s = df.loc[(df['ORF_type'] == 'super_uORF') & (df['status'] == 'translating'), 'start_codon'].tolist()

In [239]:
collections.Counter(s)

Counter({'AAG': 525,
         'ACG': 305,
         'AGG': 858,
         'ATA': 124,
         'ATC': 265,
         'ATG': 298,
         'ATT': 217,
         'CTG': 904,
         'GTG': 679,
         'TTG': 328})